

# Manually collecting the data
1. In the jupyter notebook root folder, create a new folder called "omxs30_data"
2. Go to http://www.nasdaqomxnordic.com/aktier/historiskakurser
3. Download data for wanted stocks and place the csv files in "omxs30_data"

To fully recreate the same experiment, use the same date settings and download the same stocks.
The list of stocks and selected dates can be seen in the output of cell 2 in this notebook.

In [1]:
# technical analysis library
import talib

# sklearn to develop a model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

# easy handling of data
import pandas as pd
# to make number crunching easy and fast
import numpy as np
np.random.seed(55)

# to allow file handling
import os

# pretty print!
from pprint import pprint

# to handle dates
import ciso8601
import datetime

In [2]:
## Automatically select all csv files in a folder

# folder name
directory_name = "omxs30_data"
file_list = os.listdir(directory_name)
# Remove all files that doesn't end with .csv
csv_file_list = list(filter(lambda file_name: file_name.split(".")[-1] == "csv", file_list))
# To reproduce the very same experiment, use the same stocks, as well as the same dates.
print('Stocks (including dates) used:')
pprint(csv_file_list)
# Add directory prefix
stock_file_list = [directory_name+"/"+file_name for file_name in csv_file_list]


# Uncomment row below to test with only one stock
#stock_file_list = [directory_name+"/SWMA-1900-01-01-2017-08-03.csv"]


Stocks (including dates) used:
['ABB-1900-01-01-2017-08-03.csv',
 'ALFA-1900-01-01-2017-08-03.csv',
 'ALIV-SDB-1900-01-01-2017-08-03.csv',
 'ASSA-B-1900-01-01-2017-08-03.csv',
 'ATCO-B-1900-01-01-2017-08-03.csv',
 'AZN-1900-01-01-2017-08-03.csv',
 'BOL-1900-01-01-2017-08-03.csv',
 'ELUX-B-1900-01-01-2017-08-03.csv',
 'ERIC-B-1900-01-01-2017-08-03.csv',
 'GETI-B-1900-01-01-2017-08-03.csv',
 'HM-B-1900-01-01-2017-08-03.csv',
 'INVE-B-1900-01-01-2017-08-03.csv',
 'KINV-B-1900-01-01-2017-08-03.csv',
 'LUPE-1900-01-01-2017-08-03.csv',
 'NDA-SEK-1900-01-01-2017-08-03.csv',
 'SAND-1900-01-01-2017-08-03.csv',
 'SCA-A-1900-01-01-2017-08-03.csv',
 'SEB-A-1900-01-01-2017-08-03.csv',
 'SECU-B-1900-01-01-2017-08-03.csv',
 'SKF-B-1900-01-01-2017-08-03.csv',
 'SSAB-A-1900-01-01-2017-08-03.csv',
 'SWED-A-1900-01-01-2017-08-03.csv',
 'SWMA-1900-01-01-2017-08-03.csv',
 'TEL2-B-1900-01-01-2017-08-03.csv',
 'TELIA-1900-01-01-2017-08-03.csv',
 'VOLV-B-1900-01-01-2017-08-03.csv']


In [3]:
## FEATURE ENGINEERING
# CBBL1 = Close/BBL(20,2)
# CBBL2 = Close/BBL(5,2)
# BBW1 = BBW(20,2)
# BBW2 = BBW(5,2)
# CSMA1 = Close/SMA(200)
# CSMA2 = Close/SMA(3)
# TSMA1 = Turnover/SMA(200)
# TSMA2 = Turnover/SMA(3)
# HL = High/low
# CL = Close/low
# HC = High/Close
# CC-1 = Close(t)/Close(t-1)

# FEATURE IDEAS: Day of the week, Day of the month, Month
def featureEngineer(stock_file_list):
    df_dict = {}
    for stock_file_name in stock_file_list:
        # Get the unique stock ticker
        ticker = stock_file_name.split("-")[0].split("/")[-1]

        # Read the csv file into a pandas dataframe
        df = pd.read_csv(stock_file_name, header=1, delimiter=";", decimal=",")
        # The "Unnamed: 11" column looks like an all through NaN field. Let's remove it
        df1 = df.dropna(axis=1, how="all", inplace=False)

        # Some other values are also set to NaN. "Bid", "Ask", "Opening price" and "Average price".
        # Are those fields needed? For now, no.

        # Option 1: Remove all the columns ("Bid", "Ask" etc)
        df2 = df1.drop(["Bid", "Ask", "Opening price", "Average price"], axis=1, inplace=False)

        # Option 2: Remove all the days with any NaN value.
        # The NaN days are assumed to be connected (otherwise the integrity of the data is at risk)
        #df2 = df1.dropna(axis=0, how="any", inplace=False)
        #Remove rows where a value is <=0. All values should under normal conditions be >0.
        #df = df.loc[df['Opening price'] > 0]

        # talib needs the days in ascending order
        close = df2['Closing price']
        close = np.flipud(close)


        # Moving average (unweighted rolling mean) on close price with a time period of 200 days 
        smaperiod1=200
        sma1 = talib.SMA(close, timeperiod=smaperiod1)
        csma1 = close/sma1
        csma1 = np.flipud(csma1)

        # Moving average (unweighted rolling mean) on close price with a time period of 3 days 
        smaperiod2=3
        sma2 = talib.SMA(close, timeperiod=smaperiod2)
        csma2 = close/sma2
        csma2 = np.flipud(csma2)


        # Bollinger band upper, middle, lower and width for BollingerBand(20,2)
        bbu1, bbm1, bbl1 = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
        cbbl1 = close/bbl1
        cbbl1 = np.flipud(cbbl1)

        bbw1 = bbu1/bbl1
        bbw1 = np.flipud(bbw1)


        # Bollinger band upper, middle, lower and width for BollingerBand(5,2)
        bbu2, bbm2, bbl2 = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2)

        cbbl2 = close/bbl2
        cbbl2 = np.flipud(cbbl2)

        bbw2 = bbu2/bbl2
        bbw2 = np.flipud(bbw2)


        # Moving average (unweighted rolling mean) on turnover with a time period of 200 days
        # Replace missing values with mean
        df2['Turnover'].fillna(df2['Turnover'].mean())
        turnover = df2['Turnover']
        turnover = np.flipud(turnover)

        turnoverperiod1=200
        turnover1 = talib.SMA(turnover, timeperiod=turnoverperiod1)
        turnover1 = np.array(turnover1)
        tsma1 = turnover/turnover1
        tsma1 = np.flipud(tsma1)

        # Moving average (unweighted rolling mean) on turnover with a time period of 3 days
        turnoverperiod2=3
        turnover2 = talib.SMA(turnover, timeperiod=turnoverperiod2)
        turnover2 = np.array(turnover2)
        tsma2 = turnover/turnover2
        tsma2 = np.flipud(tsma2)

        # What's the closing price 1 day earlier?
        closePrev = close/np.concatenate(([np.nan],close[:-1]))
        closePrev = np.flipud(closePrev)

        ## LABEL
        # CC1 = Close(t+1)/Close(t) <---- outcome
        # What's the closing price 1 day later?
        close1 = np.concatenate((close[1:],[np.nan]))/close
        close1 = np.flipud(close1)


        # create a dataframe using the features created above
        features_df = pd.DataFrame({"CSMA1":csma1, "CSMA2":csma2,
                                    #"TSMA1":tsma1, "TSMA2":tsma2, # <-- caused a bug, not all dates were present
                                    "CBBL1":cbbl1, "BBW1":bbw1,
                                    "CBBL2":cbbl2, "BBW2":bbw2,
                                    "CC-1":closePrev,
                                    "CC1":close1,
                                   })

        features_df['HL'] = df2['High price'].as_matrix()/df2['Low price'].as_matrix()
        features_df['CL'] = df2['Closing price'].as_matrix()/df2['Low price'].as_matrix()
        features_df['HC'] = df2['High price'].as_matrix()/df2['Closing price'].as_matrix()

        # add the feature dataframe to the existing dataframe
        df3 = pd.concat([df2, features_df], axis=1)

        # Remove days with NaN fields
        # WARNING: This removes any field NaN is present. E.g., CC1 and CC-1 contains some NaN padding
        df4 = df3.dropna(axis=0, how="any", inplace=False)


        df_dict[ticker] = df4    
    return df_dict

df_dict = featureEngineer(stock_file_list)

In [4]:
## Put all the data in one single dataframe (instead of separating them per stock)
all_stocks_df = pd.DataFrame()
for stockName, df_entry in df_dict.items():
    all_stocks_df = pd.concat([all_stocks_df, df_entry], axis=0)

In [5]:
## create test set 2
# get the date which is testset2_nr_of_days before last date
testset2_nr_of_days = 365
last_date = all_stocks_df['Date'].max()
testset2_beginning_datetime = ciso8601.parse_datetime(last_date)-datetime.timedelta(days=testset2_nr_of_days)
testset2_beginning = str(testset2_beginning_datetime)[:-9]

# separate the test set 2 data from the rest of the dataset
testset2_df = all_stocks_df[all_stocks_df['Date'] > testset2_beginning]
rest_df = all_stocks_df[all_stocks_df['Date'] <= testset2_beginning]
print("Test set 2 is %.2f" % (len(testset2_df)/len(all_stocks_df)*100) + "% of the whole dataset")

Test set 2 is 4.44% of the whole dataset


In [6]:
# Remove columns that shouldn't be used in this model
testset2_cc1 = testset2_df.pop("CC1")
testset2_date = testset2_df.pop("Date")
testset2_df2 = testset2_df.drop(["Closing price", "High price", "Low price", "Total volume", "Turnover", "Trades"], inplace=False, axis=1)

rest_df2 = rest_df.drop(["Closing price", "High price", "Low price", "Total volume", "Turnover", "Trades"], inplace=False, axis=1)




In [7]:
separateByDate = True # to avoid problems caused by correlation of stocks during the same date

# split into a training set and a test set
if (separateByDate):
    train_df_dates, testset1_df_dates = train_test_split(list(rest_df['Date'].unique()), test_size=0.2, random_state=42)
    train_df =  rest_df2[rest_df['Date'].isin(train_df_dates)]
    testset1_df =  rest_df2[rest_df['Date'].isin(testset1_df_dates)]
else:
    train_df, testset1_df = train_test_split(rest_df2, test_size=0.2, random_state=42)

In [8]:
## Close to close threshold. 1.005 means >0.5% returns True, otherwise False
ccThres = 1.005

# Remove the label (outcome)
y_cc1 = train_df.pop("CC1")
# Date might be needed later
train_date = train_df.pop("Date")
# Convert outcome to binary outcome: More than 0.5% close-to-close return or not
y = y_cc1>ccThres
# The rest of train_df should be features. Set this as training set X
X = train_df

# For later testing
y_testset1_cc1 = testset1_df.pop("CC1")
testset1_date = testset1_df.pop("Date")
y_testset1 = y_testset1_cc1>ccThres
X_testset1 = testset1_df

https://github.com/mbernico/CS570/blob/master/.ipynb_checkpoints/Random_Forests_%3D%3D_Awesome-checkpoint.ipynb
#### Thanks Mike Bernico for an excellent random forest tutorial!

In [9]:
fit_model = True # fit model or load it from file?
if (fit_model):
    model = RandomForestRegressor(n_estimators=2000, 
                                  oob_score=True, 
                                  n_jobs=-1, 
                                  random_state=42, 
                                  max_features="auto", 
                                  min_samples_leaf=10)
    model.fit(X, y)
else:
    model = joblib.load('rf_model.pkl')

In [18]:
# random forest model outputs a value between 0 and 1.
# 1 means that all the trees agree that it should be more than a 0.5% gain
# 0 means that all the trees agree that it should be equal to or less than 0.5% gain
thres = 0.5

## TEST SET 1

y_test_prediction = list(model.predict(X_testset1))
y_test_actual = list(y_testset1)
y_test_cc1_actual = list(y_testset1_cc1)

test_set_prediction_df = pd.DataFrame({"Output":y_test_prediction,
                                        "Date":testset1_date,
                                        "Outcome":y_test_cc1_actual})


bar = (test_set_prediction_df["Outcome"].mean()*100-100)
bsd = (test_set_prediction_df["Outcome"].std()*100)
bsr = bar/bsd
print('Benchmark Average Return (AR): %.2f' % bar + "%")
print('Benchmark Standard Deviation (SD): %.2f' % bsd + "%")
print('Benchmark Sharpe Ratio (SR): %.2f' % bsr)
print()
# m
test_set_buy_signals = test_set_prediction_df[test_set_prediction_df["Output"] > thres]
ar = (test_set_buy_signals["Outcome"].mean()*100-100)
sd = (test_set_buy_signals["Outcome"].std()*100)
sr = ar/sd
print('Average Return (AR): %.2f' % ar + "%")
print('Standard Deviation (SD): %.2f' % sd + "%")
print('Sharpe Ratio (SR): %.2f' % sr)
print()
print('Nr of buy signals %i' % len(test_set_buy_signals))
print('Nr of days with one or more buy signals %i' % len(test_set_buy_signals["Date"].unique()))
print('Total nr of days %i' % len(test_set_prediction_df))

Benchmark Average Return (AR): 0.14%
Benchmark Standard Deviation (SD): 2.23%
Benchmark Sharpe Ratio (SR): 0.06

Average Return (AR): 0.47%
Standard Deviation (SD): 3.75%
Sharpe Ratio (SR): 0.13

Nr of buy signals 1349
Nr of days with one or more buy signals 659
Total nr of days 28238


In [19]:
#TEST SET 2 - the real deal

test_set_output = model.predict(testset2_df2)
test_set_prediction_df = pd.DataFrame({"Output":test_set_output,
                                        "Date":testset2_date,
                                        "Outcome":testset2_cc1})

bar = (test_set_prediction_df["Outcome"].mean()*100-100)
bsd = (test_set_prediction_df["Outcome"].std()*100)
bsr = bar/bsd
print('Benchmark Average Return (AR): %.2f' % bar + "%")
print('Benchmark Standard Deviation (SD): %.2f' % bsd + "%")
print('Benchmark Sharpe Ratio (SR): %.2f' % bsr)
print()
# m
test_set_buy_signals = test_set_prediction_df[test_set_prediction_df["Output"] > thres]
ar = (test_set_buy_signals["Outcome"].mean()*100-100)
sd = (test_set_buy_signals["Outcome"].std()*100)
sr = ar/sd
print('Average Return (AR): %.2f' % ar + "%")
print('Standard Deviation (SD): %.2f' % sd + "%")
print('Sharpe Ratio (SR): %.2f' % sr)
print()
print('Nr of buy signals %i' % len(test_set_buy_signals))
print('Nr of days with one or more buy signals %i' % len(test_set_buy_signals["Date"].unique()))
print('Total nr of days %i' % len(test_set_prediction_df))

Benchmark Average Return (AR): 0.06%
Benchmark Standard Deviation (SD): 1.66%
Benchmark Sharpe Ratio (SR): 0.04

Average Return (AR): 0.34%
Standard Deviation (SD): 1.74%
Sharpe Ratio (SR): 0.19

Nr of buy signals 63
Nr of days with one or more buy signals 53
Total nr of days 6578


In [12]:
# testing the compounded interest for the signals produced by test set 2
# some of the signals are on the same date, which is taken care of by
# allocating an equal portion of the fictive portfolio

date_sorted_buy_signals = test_set_buy_signals.sort_values(by="Date", ascending=False)

trade_results = []
non_clustered_signal_results = []
clustered_signal_results = []
for date in date_sorted_buy_signals["Date"].unique():
    trades_on_date = test_set_buy_signals[test_set_buy_signals["Date"] == date]
    allocation = len(trades_on_date)
    individual_trade_results = []
    for index, trade in trades_on_date.iterrows():
        individual_trade_results.append(trade["Outcome"]/allocation)
        if (len(trades_on_date) > 1):
            clustered_signal_results.append(trade["Outcome"])
        else:
            non_clustered_signal_results.append(trade["Outcome"])
    trade_results.append(np.sum(individual_trade_results))
trade_results
compounded = 1.0
for result in trade_results:
    compounded *= result
compounded

clustered_outcome_mean = (np.mean(clustered_signal_results)*100-100)
non_clustered_outcome_mean = (np.mean(non_clustered_signal_results)*100-100)

print("Clustered mean %.2f" % clustered_outcome_mean + "%")
print("Non clustered mean %.2f" % non_clustered_outcome_mean + "%")

#print(np.max(clustered_signal_results)*100-100)
#print(np.min(clustered_signal_results)*100-100)
#print(np.max(non_clustered_signal_results)*100-100)
#print(np.min(non_clustered_signal_results)*100-100)


Clustered mean 1.53%
Non clustered mean -0.18%


In [13]:
# run this to see the difference between
# non clustered and clustered buy signals
import plotly.offline as py
import plotly.graph_objs as go


py.init_notebook_mode(connected=True)


trace1 = go.Histogram(
    x=np.array(clustered_signal_results)*100-100,
    opacity=0.75,
    histnorm='probability',
    name='Clustered signals',
    xbins=dict(
        start=-10.0,
        end=10.0,
        size=1
    )
)
trace2 = go.Histogram(
    x=np.array(non_clustered_signal_results)*100-100,
    opacity=0.75,
    histnorm='probability',
    name='Non clustered signals',
    xbins=dict(
        start=-10.0,
        end=10.0,
        size=1
    )
)

data = [trace1, trace2]
layout = go.Layout(barmode='overlay', title='Distribution of buy signal outcomes', 
                  xaxis=dict(
        title='Outcome [%]',
        ),
                  yaxis=dict(
        title='Probability',
       ))
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='overlaid histogram')